<a href="https://colab.research.google.com/github/jaitrasaketh/ACMQuant/blob/main/JaitraSakethACMQuant_Round2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Task :

> Indented block


Simple Moving Average Crossover Strategy

> Indented block


Objective: Implement a basic algorithmic trading strategy that uses moving average crossovers to
generate buy and sell signals. Evaluate the strategy by backtesting on historical data.

0. Data collection:
a. Use yFinance to download 5-10 years of historical OHLC data for Nifty50 stocks
b. Alternately can use NIFTY index data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/^NSEI.csv"
data=pd.read_csv(path);



I. Data Preprocessing:
A. Handle missing values


B. Add technical indicators like trading volume, volatility, etc.
C. Engineer features like daily returns, log returns, lagged prices

In [ ]:
# A. Handle missing values
data.dropna(inplace=True)

# B. Add technical indicators like trading volume, volatility, etc.

data['Volatility'] = data['Close'].pct_change().rolling(window=20).std()
data['Volume_MA'] = data['Volume'].rolling(window=20).mean()

#Relative Strength Index (RSI) Calculation
def calculate_rsi(data, window=14):
    # Calculate price changes, gains, and losses
    delta = data['Close'].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Calculate average gain and loss over the specified window
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    # Calculate relative strength (RS) and RSI
    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))

    return data

# Apply RSI
data = calculate_rsi(data)

# C. Engineer features like daily returns, log returns, lagged prices
data['Daily_Return'] = data['Close'].pct_change()
data['Log_Return'] = np.log(1 + data['Daily_Return'])
data['Lagged_Close'] = data['Close'].shift(1)


II. Strategy Development:
A. Calculate 50-day and 200-day Simple Moving Averages
B. Generate buy signal when 50-SMA crosses above 200-SMA
C. Generate sell signal when 50-SMA crosses below 200-SMA

In [ ]:
#A
data['50SMA'] = data['Close'].rolling(window=50).mean()
data['200SMA'] = data['Close'].rolling(window=200).mean()

# Create a 'Signal' column
data['Signal'] = 0  # Initialize with zeros

# 1 is buy signal, -1 is sell signal

# Set signals based on conditions
data.loc[data['50SMA'] > data['200SMA'], 'Signal'] = 1  # B
data.loc[data['50SMA'] < data['200SMA'], 'Signal'] = -1  # C

# 'Position' column in data
data['Position'] = data['Signal'].shift(1)


III. Backtesting Engine:
A. Simulate trades by applying signals on test data
B. Track quantities like invested capital, portfolio value, returns

In [ ]:
def backtest(data, initial_capital=100000):
    #DataFrame to hold trading positions
    positions = pd.DataFrame(index=data.index).fillna(0.0)
    positions['Position'] = data['Position']  #based on input data

    #DataFrame to track portfolio metrics
    portfolio = pd.DataFrame(index=data.index).fillna(0.0)
    portfolio['Cash'] = initial_capital - (positions['Position'].diff() * data['Close']).cumsum()
    portfolio['Holdings'] = (positions['Position'] * data['Close'])
    portfolio['Total'] = portfolio['Cash'] + portfolio['Holdings']
    portfolio['Returns'] = portfolio['Total'].pct_change()

    return portfolio

# Perform backtesting
portfolio = backtest(data, initial_capital=100000)
print(portfolio)

               Cash      Holdings          Total   Returns
399             NaN           NaN            NaN       NaN
400             NaN      0.000000            NaN       NaN
401   100000.000000      0.000000  100000.000000       NaN
402   100000.000000      0.000000  100000.000000  0.000000
403   100000.000000      0.000000  100000.000000  0.000000
...             ...           ...            ...       ...
1230   80608.654299  21737.599609  102346.253908  0.003776
1231   80608.654299  21522.099609  102130.753908 -0.002106
1232   80608.654299  21725.699219  102334.353518  0.001994
1233   80608.654299  21697.449219  102306.103518 -0.000276
1234   80608.654299  21853.800781  102462.455080  0.001528

[833 rows x 4 columns]


IV. Performance Evaluation:
A. Calculate metrics like annual return, Sharpe ratio, max drawdown
B. Compare strategy returns to Nifty50 returns

In [ ]:


#Calculate annual return
portfolio_initial_value = portfolio['Total'].iloc[0]
portfolio_final_value = portfolio['Total'].iloc[-1]
annual_return = ((portfolio_final_value / portfolio_initial_value) ** (252 / len(data))) - 1

# Calculate Sharpe ratio
sharpe_ratio = data['Daily Portfolio Returns'].mean() / data['Daily Portfolio Returns'].std() * np.sqrt(252)

# Calculate maximum drawdown
max_drawdown = (portfolio['Total'] / portfolio['Total'].cummax() - 1).min()

# Compare strategy returns to Nifty50 returns
nifty_initial_value = data['Close'].iloc[0]
nifty_final_value = data['Close'].iloc[-1]
nifty_return = ((nifty_final_value / nifty_initial_value) ** (252 / len(data))) - 1


V. Visualization:
A. Interactive charts showing equity curve, signals
B. Use matplotlib

In [ ]:
# Visualize equity curve
plt.figure(figsize=(14, 8))
plt.plot(portfolio['Total'], label='Portfolio Value', color='blue')
plt.title('Equity Curve')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()

# Plot interactive chart with buy/sell signals
plt.figure(figsize=(14, 8))
plt.plot(data.index, data['Close'], label='Close Price', color='black')
plt.scatter(data[data['Position'] == 1].index, data['Close'][data['Position'] == 1], marker='^', color='g', label='Buy Signal')
plt.scatter(data[data['Position'] == -1].index, data['Close'][data['Position'] == -1], marker='v', color='r', label='Sell Signal')




plt.title('Buy and Sell Signals ')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()

NameError: name 'plt' is not defined

VI. Improvements (Optional)
A. Parameter tune SMA periods
B. Combine with other indicators like RSI

In [ ]:
#B already combined RSI technical indicator